# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Prompt Engineering Best Practices: Building Chatbots </center>

# <center style="font-family: consolas; font-size: 25px; font-weight: bold;">  Prompt Engineering for Instruction-Tuned LLMs </center>
***

One of the compelling aspects of utilizing a large language model lies in its capacity to effortlessly construct a personalized chatbot and leverage it to craft your very own chatbot tailored to various applications. 

In the forthcoming tutorial, we delve deep into the OpenAI chat completions format, unraveling its nuances and intricacies to provide you with a comprehensive understanding. 

Armed with this knowledge, you'll embark on an enlightening journey towards constructing your very own chatbot from the ground up. Through step-by-step guidance and practical demonstrations, you'll unlock the potential to shape conversational experiences that resonate with your audience, driving engagement and efficiency in your chosen domain.


 #### <a id="top"></a>

# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Setting Working Environment & Getting Started </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. Understanding Messages Roles </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3. Build a Customized Chatbot </a></li>     
</ul>
</div>

***




<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Setting Working Environment & Getting Started </b></div>




We will use the OpenAI Python library to access the OpenAI API. You can use this Python library using pip like this:

In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


Next, we will import OpenAI and then set the OpenAI API key as a secret key. You can get one of these API keys from the OpenAI website. Setting this as an environment variable is better to keep it safe if you share your code. We will use OpenAI's chatGPT GPT 3.5 Turbo model, and the chat completions endpoint.

In [ ]:
import os
import utils
import panel as pn
from openai import OpenAI
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI

# ==============================
# 🔑 Load environment variables
# ==============================
env_values = dotenv_values(".env")

openai_api_key = env_values.get("OPENAI_API_KEY")
openai_api_base = env_values.get("OPENAI_API_BASE")
openai_api_name = env_values.get("OPENAI_API_NAME")

# Initialize Clients
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base
)

# Mock moderation for OpenRouter compatibility
if openai_api_base and "openrouter.ai" in openai_api_base:
    class MockResult(dict):
        def __getattr__(self, name): return self.get(name)
    class MockModeration(dict):
        def __init__(self):
            res = MockResult(flagged=False, categories={}, category_scores={})
            super().__init__(results=[res])
        @property
        def results(self): return self["results"]
    client.moderations.create = lambda *args, **kwargs: MockModeration()

chat = ChatOpenAI(
    model_name=openai_api_name, 
    openai_api_key=openai_api_key, 
    openai_api_base=openai_api_base,
    temperature=0.7
)
pn.extension()


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

Let's define two helper functions the first one is one that we've been using throughout all the videos and it's the get_completion function if you kind of look at it we give a prompt but then inside the function what we're doing is putting this prompt into what looks like some kind of user message and this is because the chatGPT model is a chat model which means it's trained to take a series of messages as input and then return model generated messages output. Therefore the user message is the input and then the assistant messages are the output.

In [ ]:
# ==============================
# 🔧 Helper Functions
# ==============================
def get_completion(prompt, model=openai_api_name, temperature=0, max_tokens=500):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

We also are going to use a new helper function instead of defining a single prompt as input and getting a single completion as a response. We are going to pass in a list of messages and these messages can be kind of from a variety of different roles.

In [ ]:
def get_completion_from_messages(messages, model=openai_api_name, temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Understanding Messages Roles </b></div>


There are two main message types we will use. The first message is a system message which gives an overall instruction to the LLM. After this message, we have turns between the user and the assistant and this continues to go on. If you have used the web interface of chatGPT then your messages are the user messages and then ChatGPT's messages are the assistant messages. 

Therefore the system message helps to set the behavior and Persona of the assistant and it acts as a high-level instruction for the conversation. You can kind of think of it as whispering in the assistant's ear and guiding its responses without the user being aware of the system message. So as the user, if you've ever used ChatGPT you probably don't know what's in the ChatGPT system message. 

The benefit of the system message is that it allows you the developer to frame the conversation without making the request itself part of the conversation. So you can guide the assistant and whisper in its ear and guide its responses without making the user aware of it. 
Let's try to use these messages in a practical conversation to have a better understanding. We will use the  new helper function to get the completion from the messages.

In [5]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

The system message says you are an assistant that speaks like Shakespeare. This is our description to the assistant on how it should behave. The first user message is "Tell me a joke" and the next system message is "Why did the chicken cross the road" and then the final user message is "I don't know". If we run this response we will get to the other side of the joke.

In [6]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To reach the other side, fair friend! Forsooth, ‘tis but a jest of simple wit. Wouldst thou like to hear another, perchance?


Let's take  another example in which the system message will be " You are a friendly chatbot" and the first user message is "Hi, my name is Youssef". So let's execute this to get the assistant message which will be "Hello Youssef! It's nice to meet you. How are you doing today?"

In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Youssef'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Youssef! It’s great to meet you. How can I assist you today?


Let's try another example in which the system message is "You are a friendly chatbot" and the user message is "Yes, can you remind me, What is my name?" 
Let's get the response to this message.

In [8]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I don’t have access to personal data about users unless you share it with me in the conversation. If you tell me your name, I’ll be happy to remember it for the rest of our chat!


You can see the model doesn't know my name so each conversation with a language model is a standalone interaction which means that you must provide all relevant messages for the model to draw from in the current conversation.
Therefore  if you want the model to remember earlier parts of the conversation you must provide the earlier exchanges in the input to the model and so we'll refer to this as context. Let's try this now after we have given the context that the model needs which is my name in the previous messages. We will ask the same question so we'll ask what my name is and the model can respond because it has all of the contacts it needs

In [9]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Youssef'},
{'role':'assistant', 'content': "Hi Youssef! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Youssef! How can I assist you further?


<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b>3. Build a Customized  Chatbot </b></div>


Now you're going to build our chatbot this chatbot is going to be called Pizzabot and we're going to automate the collection of user prompts and assistant responses to build this Pizzabot which is going to take orders at a pizza restaurant.
First, we're going to define the collect_messages helper function which will collect our user messages so we can avoid typing them in by hand in the same way that we did above. So it is going to collect prompts from a user interface that will build below and then append it to a list called context and then it will call the model with that context every time.

The model response is then also added to the contacts so the model message is added to the context the user message is added to the context and so on.  Therefore  it just kind of grows longer and longer this way the model has the information it needs to determine what to do next.

In [10]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages
panels = [] # collect display

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

Now we will set up and run the UI to display the Pizzabot here's the context and it contains the system message that contains the menu note that every time we call the language model we're going to use the same context and the context is building up over time. 
 The system message to the Pizzabot service is designed to collect orders for a pizza restaurant. We first asked the system to greet the customer and and collect the order and ask if it's a pickup or delivery and you wait to collect the entire order. Then you will summarize it and check for a final time if the customer wants to add anything else if it's a delivery you can ask for an address. Finally, you collect the payment and make sure to clarify all options extras, and sizes to uniquely identify the item from the menu. You respond in a short very conversational friendly style the menu includes and then here we have the menu
After executing it we can see the chatbot GUI below.

In [11]:
import os
import utils
import panel as pn
from openai import OpenAI
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI

# ==============================
# 🔑 Load environment variables
# ==============================
env_values = dotenv_values(".env")

openai_api_key = env_values.get("OPENAI_API_KEY")
openai_api_base = env_values.get("OPENAI_API_BASE")
openai_api_name = env_values.get("OPENAI_API_NAME")

# Initialize Clients
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base
)

# Mock moderation for OpenRouter compatibility
if openai_api_base and "openrouter.ai" in openai_api_base:
    class MockResult(dict):
        def __getattr__(self, name): return self.get(name)
    class MockModeration(dict):
        def __init__(self):
            res = MockResult(flagged=False, categories={}, category_scores={})
            super().__init__(results=[res])
        @property
        def results(self): return self["results"]
    client.moderations.create = lambda *args, **kwargs: MockModeration()

chat = ChatOpenAI(
    model_name=openai_api_name, 
    openai_api_key=openai_api_key, 
    openai_api_base=openai_api_base,
    temperature=0.7
)
pn.extension()

# ==============================
# 🔧 Helper Functions
# ==============================
def get_completion(prompt, model=openai_api_name, temperature=0, max_tokens=500):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model=openai_api_name, temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

C:\Users\Administrator\AppData\Local\Temp\ipykernel_5264\312141790.py:41: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


I am going to say to the chatbot "Hi, I would like to order a pizza" and the assistant will respond "Great choice! Which pizza would you like to order? We have pepperoni pizza for $12.95, cheese pizza for $10.95, and eggplant pizza for $11.95."

Next, we will order a medium cheese Pizza, and here is the response:

We can alos order some sides such as water and fries:

Now we will finalize the order by saying no I do not want anything else and choose the delivery and payment method:

Finally, we can ask the model to create a  Json summary for our order that we can send to the order system based on the conversation. We will append another system message in which we give it instructions to  create a Json summary of the previous food order items and the price for each item. The field should include the pizza ordered, the list of toppings, the list of drinks, the list of sides, and finally the total price.
We will use a lower temperature because, for this kind of task, we want the output to be fairly predictable for a conversational agent.

In [12]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Sure! Let's get started with your pizza order! 🍕 

What kind of pizza would you like? We have pepperoni, cheese, and eggplant. And what size do you prefer? We have small, medium, and large options!


In summary, we have it you've built your very own order chatbot. Feel free to kind of customize it yourself and play around with the system message to change the behavior of the chatbot and make it act with different personas and different knowledge.